In [1]:
import os
import sys

In [2]:
os.chdir('/home/au643300/DataHandling/')
sys.path.append('/home/au643300/DataHandling/')
os.environ['HDF5_USE_FILE_LOCKING']="FALSE"

In [3]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, as_completed,wait,fire_and_forget
import glob
from src.data.tonetCDF import to_netcdf
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from src.interim import read_valdata
import numpy as np
import importlib
import dask
import zarr
import time

In [4]:
sns.set_theme()
dask.config.set({"distributed.comm.timeouts.tcp": "50s"})
dask.config.set({'distributed.comm.retry.count':3})
dask.config.set({'distributed.comm.timeouts.connect':'25s'})
dask.config.set({"distributed.worker.use-file-locking":False})

In [5]:
cluster=SLURMCluster(cores=6,
                     memory="22GiB",
                     queue='q64',
                     walltime='0-01:00:00',
                     local_directory='/scratch/$SLURM_JOB_ID',
                     interface='ib0',
                     scheduler_options={'interface':'ib0'},
                     extra=['--resources mem=15',"--lifetime", "55m", "--lifetime-stagger", "2m"]
                    )
                    

In [6]:
client=Client(cluster)

In [7]:
cluster.adapt(minimum=1,maximum=1)

In [8]:
client

Client Scheduler: tcp://10.100.8.23:38753 Dashboard: http://10.100.8.23:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [9]:
files=glob.glob("/home/au643300/NOBACKUP/interim/*nc")
files.sort()
chunks = [files[x:x+20] for x in range(0, len(files), 20)]

In [ ]:
ds=xr.open_mfdataset(files[0],parallel=True,combine='nested',concat_dim='time')
ds.to_zarr("/home/au643300/NOBACKUP/data.zarr",consolidated=True,compute=True,mode='w')


for file in files[1:]:
    ds=xr.open_mfdataset(file,parallel=True,combine='nested',concat_dim='time')
    ds.to_zarr("/home/au643300/NOBACKUP/data.zarr", append_dim="time",consolidated=True,compute=True)